In [4]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [5]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
MODEL= 'gpt-4o-mini'

In [6]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [7]:
openai = OpenAI()


In [8]:
system_message = "You are a helpful assistant for an apple store."
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [9]:
apple_price = {"macbook": "$799", "macbook pro": "$2500", "ipad air": "$700", "ipad pro": "$1300"}

In [10]:
def get_product_price(product_name):
    print(f"Tool get_product_price called for {product_name}")
    name = product_name.lower()
    # print()
    return apple_price.get(name, "Unknown")

In [11]:
price_function = {
    "name": "get_product_price",
    "description": "Get the price of an apple product. Call this whenever you need to know the apple product price, for example when a customer asks 'How much is a macbook air'",
    "parameters": {
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "The price of apple product that the customer wants to buy and know",
            },
        },
        "required": ["product_name"],
        "additionalProperties": False
    }
}


In [12]:
tools = [{'type':'function',"function":price_function}]

In [13]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    product_name = arguments.get('product_name')
    price  = get_product_price(product_name)

    response = {
        'role': 'tool',
        'content': json.dumps({'product_name':product_name, 'price':price}),
        'tool_call_id': tool_call.id
    }
    return response,product_name


In [14]:
def chat(message,history):
    messages = [{'role':'system','content':system_message}]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        response,product_name = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content
        

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
